In [2]:
import pandas as pd
import numpy as np

### Carga, unión y filro de BD enero, febrero, marzo

In [ ]:
# # Instala openpyxl si no está instalado
# %pip install openpyxl


def leer_excel(file_path):
    """
    Lee un archivo Excel y devuelve un DataFrame en pandas.

    :param file_path: Ruta al archivo Excel.
    :return: DataFrame con los datos del archivo Excel.
    """
    return pd.read_excel(file_path)


# Combina los DataFrames de enero, febrero y marzo
def CombinarDataFrames(df_enero, df_febrero, df_marzo):
    """
    Combina tres DataFrames en uno solo.

    :param df_enero: DataFrame de BD enero.
    :param df_febrero: DataFrame de BD febrero.
    :param df_marzo: DataFrame de BD marzo.
    :return: DataFrame combinado.
    """
    return pd.concat([df_enero, df_febrero, df_marzo], ignore_index=True)


def FiltrarDataFramePorFecha(df, dia, mes, anhio):
    """
    Filtra un DataFrame por día, mes y año.

    :param df: DataFrame a filtrar.
    :param dia: Día del mes.
    :param mes: Mes del año (Enero, Febrero o Marzo).
    :param anhio: Año. fijo en 2021.
    :return: DataFrame filtrado por fecha.
    """
    # Filtrar las filas por día, mes y año con inputs del usuario
    dia = int(input("Ingrese el día (1-31): "))
    mes = int(input("Ingrese el mes (1: Enero, 2: Febrero, 3: Marzo): "))
    anhio = 2021  # Cambia el anhio según sea necesario

    df["Fecha corte"] = pd.to_datetime(df["Fecha corte"], errors="coerce")

    return df[
        (df["Fecha corte"].dt.day == dia)
        & (df["Fecha corte"].dt.month == mes)
        & (df["Fecha corte"].dt.year == anhio)
    ]


def FiltrarColumnas(df):
    """
    Filtra el DataFrame para mostrar solo las columnas deseadas.

    :param df: DataFrame a filtrar.
    :return: DataFrame filtrado con las columnas deseadas.
    """
    # Filtrar las columnas deseadas
    columnas_deseadas = [
        "Fecha corte",
        "Tipo Entidad",
        "Nombre Entidad",
        "Valor fondo al cierre del día t",
        "Núm. Invers.",
    ]
    return df[columnas_deseadas]


def ExportarDataFrame(df, file_name):
    """
    Exporta un DataFrame a un archivo Excel.

    :param df: DataFrame a exportar.
    :param file_name: Nombre del archivo de salida.
    """
    df.to_excel(file_name, index=False)


if __name__ == "__main__":
    df_enero = leer_excel(file_path="BD enero.xlsx")
    df_febrero = leer_excel(file_path="BD febrero.xlsx")
    df_marzo = leer_excel(file_path="BD marzo.xlsx")
    df_combinado = CombinarDataFrames(df_enero, df_febrero, df_marzo)
    df_filtrado = FiltrarDataFramePorFecha(df_combinado, dia=1, mes=1, anhio=2021)
    df_filtrado = FiltrarColumnas(df_filtrado)
    ExportarDataFrame(df_filtrado, "Macro.xlsx")

### Descarga del pdf de Ficha Técnica Vigente

In [ ]:
import requests
"""
El enlace para la ficha técnica de BBVA Colombia, posiblemente 
cambie con el tiempo, por lo que es recomendable verificar la URL actual 
y la del próximo mes, porque seguramente, cambie la fecha 20250507,
lo que habilita un ciclo entre los días del mes y poder descargar la ficha tecnica vigente.
"""

url = "https://fidocs.bbva.com/documents/CCAPAISCB_FTCOL_20250507_COL_SPA_0.pdf"  # Reemplaza URL
nombre_archivo = "ficha_tecnica.pdf"

try:
    # Realizar solicitud HTTP
    respuesta = requests.get(url, stream=True)
    respuesta.raise_for_status()  # Verificar errores

    # Guardar el contenido en un archivo PDF
    with open(nombre_archivo, 'wb') as archivo:
        for chunk in respuesta.iter_content(chunk_size=8192):
            archivo.write(chunk)
    
    print(f"PDF descargado: {nombre_archivo}")

except Exception as e:
    print(f"Error: {e}")

PDF descargado: ficha_tecnica.pdf


## Análisis de sensibilidad

In [ ]:
# # Cargar Industria Local FICs.xlsm
# df_detalle = pd.read_excel("Industria Local FICs.xlsm", sheet_name="Detalle Fondos",header=1)
# # Eliminar las dos primeras columnas de df_detalle
# df_detalle = df_detalle.iloc[:, 2:]
# # Filtrar la donde la columna 'NOMBRE ADMINISTRADORA' sea igual a 'FIDUCIARIA BANCOLOMBIA'
# df_detalle = df_detalle[df_detalle['NOMBRE ADMINISTRADORA'] == 'FIDUCIARIA BANCOLOMBIA']

df_Resultados = pd.read_excel(
    "Industria Local FICs.xlsm", sheet_name="tblResultadoReporte", header=1
)
df_Resultados = df_Resultados.copy()
# # Columnas de df_Resultados
# print(df_Resultados.columns.tolist())

# Eliminar las filas con nulos en la columna 'COMISIÓN'
df_Resultados = df_Resultados.dropna(subset=["COMISIÓN"])

# Eliminar valores que no sean de tipo numérico en la columna 'COMISIÓN'
df_Resultados = df_Resultados[
    pd.to_numeric(df_Resultados["COMISIÓN"], errors="coerce").notnull()
]
# Convertir la columna 'COMISIÓN' a tipo numérico
df_Resultados["COMISIÓN"] = pd.to_numeric(df_Resultados["COMISIÓN"], errors="coerce")
# Ordenar el DataFrame por la columna 'COMISIÓN' de forma ascendente
df_Resultados = df_Resultados.sort_values(by="COMISIÓN", ascending=True)


# Analisis de sensiblidad de las comisiones con un monto de 1000000 durante 10 años
# Monto de inversión y DataFrame para análisis de sensibilidad con dataframe nuevo
monto_inversion = 1000000
crecimiento_anual = 0.044  # Crecimiento anual del 4.4%
df_as = pd.DataFrame()

# Crear Columnnas para el análisis de sensibilidad
df_as["YEAR"] = np.arange(0, 11)  # Años de 1 a 10
# Crear crear columna MONTO_INVERSION, Si el año es cero, el monto de inversión es el inicial, en adelante, el monto de inversión se calcula con el crecimiento anual
df_as["MONTO_INVERSION"] = [
    monto_inversion * (1 + crecimiento_anual) ** year for year in df_as["YEAR"]
]

nuevas_columnas = {}
COMISIONES_UNICAS = df_Resultados["COMISIÓN"].unique()

# Calcular los percentiles de comisión en un solo paso
percentiles = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00]
comisiones_percentiles = [np.quantile(COMISIONES_UNICAS, p) for p in percentiles]
# Redondear comisiones_percentiles a dos decimales
comisiones_percentiles = [
    round(comision / 100, 4) for comision in comisiones_percentiles
]

for comision in comisiones_percentiles:
    nuevas_columnas[f"COMISION_{round(comision*100,2)}%"] = (
        df_as["MONTO_INVERSION"] * comision
    )

# Concatenar todas las nuevas columnas al DataFrame original
df_as = pd.concat([df_as, pd.DataFrame(nuevas_columnas)], axis=1)

# Exportar df_as a un archivo Excel
# ExportarDataFrame(df_as, "Analisis_Sensibilidad_Comisiones.xlsx")

# Análisis descriptivo sólo para las comisiones comoo dataframe
df_as = df_as.drop(columns=["YEAR", "MONTO_INVERSION"])

# guardar df_as.describe().T en un dataframe
df_as_describe = df_as.describe().T

# Exportar df_as_describe a un archivo Excel
#ExportarDataFrame(df_as_describe, "Analisis_Sensibilidad_Comisiones_Descriptive.xlsx")